# Introduction
Welcome to the Pygame Modelling Workshop. You can find the track and tasks in this notebook. We assume that you have `python3.7+` with `pip` installed and you have installed the provided package `pygmodw25` with all it's dependencies in a virtual environment (venv) from which you are running this notebook. In case you have not yet done the preparatory steps, please follow the instructions [in the README file](https://github.com/mezdahun/PygameModelling25#1-prerequisites)

## Goal of the Workshop
During this workshop you will get a hands-on introduction and demonstration on modelling a typical multi-agent system using an open-source game engine (pygame). 

The main goals of this workshop is to:
1. Learn how typical agent-based models are build into simulation fameworks. (Definition of agent, simulation, bottom-up modeling)
2. Learn the code structure of a typical python package and modify it to build our own models
3. Learn about typical zonal (distance-dependent) flocking models of collective motion
4. Implement such a flocking model in pygame 
5. Interact with the model through the game engine and explore it's behavior

## Codebase
I have initialized a basic python package with 3 simple files containing the structure for our agent-based simulation within the pygmodw25 folder. Whatever you change within those files, will first need to be installed via `pip` to include in the code within this notebook. In this section we learn how the package is built and how we can change it's behavior for our purposes.

### What's inside pygmodw25?
* The `__init__.py` file tells python that this is a module (package) that (after installation) can be imported within our won code.
* In `agent.py` we describe the individual agents of our model. These you can think of as individual "animals" in a group. Here you can define their individual capabilites, appearance and behavior. This file also holds most interaction rules between agents (social interactions). Agent-based modeling is also called "bottom up", because we model collective behavior starting with the lowest building blocks and first principles (i.e., the individuals within the group). Top-down models, on the other hand, usually work with some statistical summaries about the system and their changes. Do you know about any top-down models?
* In `sims.py` we describe the world and it's temporal evolution in which our agents exist. You can think of it as the rules of the world that the agents must obey. It can also include some (but not all) interaction rules between agents that are environment specific, for example collision rules (what happens if 2 agents overlap in space and time). In other words, think of this file as what is the basic physical laws in your virtual world.
* `support.py` includes all supllementary and mathematical methods needed for the update process. These are, for example, calculation of euclidian distance, implementation of the sigmoid function, etc.

### What's inside the individual files?
In both `agent.py` and `sims.py` you will find 2 main classes. These are `AgentBase` and `Simulation`. Due to the short time we have, I prepared these with some basic functionalities. During the class we go through these shortly together. 

### How we define the agents?
Let's see some examples to what's inside the `AgentBase` file.
* the `__init__` function always initializes a given class. It gives it's base attributes, in our case each agent has an ID, certain radius (as they are circular, like pacman), color, speed and orientation in the 2D world we create. Then at the end of this fucntion we visualize the agent using pygame within the world.
* this class also has some handy function already implemented for you, for example changing it's color, moving with the cursor in the window, get reflected from the walls of the arena (if any), etc.
* the most important function of this class is the `update` method which is called in every simulation step and it defines how agents behave. This is where you have to integrate your theoretical model. In other words, this is where you calculate, in each simulation step, how the agent must change it's internal state (velocity, turning rate, position, orientation, etc.) given the state of the environment (in our case usually the state of other agents).
* The current implementation is as follows:

```python
def update(self, agents):
    """
    main update method of the agent. This method is called in every timestep to calculate the new state/position
    of the agent and visualize it in the environment
    :param agents: a list of all other agents in the environment.
    """
    if not self.is_moved_with_cursor:  # we freeze agents when we move them
        # # updating agent's state variables according to calculated vel and theta
        self.orientation += self.dt * self.dtheta
        self.prove_orientation()  # bounding orientation into 0 and 2pi
        self.velocity += self.dt * self.dv
        self.prove_velocity()  # possibly bounding velocity of agent

        # updating agent's position
        self.vx = self.velocity * np.cos(self.orientation)
        self.vy = self.velocity * np.sin(self.orientation)
        self.position[0] += self.vx
        self.position[1] -= self.vy

        # boundary conditions if applicable
        self.reflect_from_walls(self.boundary)

    # updating agent visualization
    self.draw_update()
```

This method recieves all other agents data in the virtual world (this is the input of the method) and updates the focal agent (to which this method belongs to) accordingly. Currently, the agent does not interact with others and just moves in a straight line.

### How we define the simulation?
Similarly, some basic simulation features (such as building a rectangular world and adding agents to it) has been implemented for your convenience in the `Simulation` class of the `sims.py` file. At the end of the `__init__` method we initialize the pygame environment (the game engine) and create the agents within the world:

```python
# Initializing pygame
pygame.init()

# pygame related class attributes
self.agents = pygame.sprite.Group()
# Creating N agents in the environment
self.create_agents()
self.screen = pygame.display.set_mode([self.WIDTH + 2 * self.window_pad, self.HEIGHT + 2 * self.window_pad])
self.clock = pygame.time.Clock()
```

Within this class the most important method that iterates through timesteps and updates the agents within the world is the `start` method. Here is the current implementation:

```python
def start(self):

    start_time = datetime.now()
    print(f"Running simulation start method!")

    print("Starting main simulation loop!")
    # Main Simulation loop until dedicated simulation time
    while self.t < self.T:

        events = pygame.event.get()
        # Carry out interaction according to user activity
        self.interact_with_event(events)

        if not self.is_paused:

            if self.physical_collision_avoidance:
                # ------ AGENT-AGENT INTERACTION ------
                # Check if any 2 agents has been collided and reflect them from each other if so
                collision_group_aa = pygame.sprite.groupcollide(
                    self.agents,
                    self.agents,
                    False,
                    False,
                    within_group_collision
                )
                collided_agents = []
                # Carry out agent-agent collisions and collecting collided agents for later (according to parameters
                # such as ghost mode, or teleportation)
                for agent1, agent2 in collision_group_aa.items():
                    self.agent_agent_collision(agent1, agent2)

            # Updating behavior of all agents within the simulation
            for agent in self.agents:
                agent.update(self.agents)

            # Update agents according to current visible obstacles
            self.agents.update(self.agents)

            # move to next simulation timestep
            self.t += 1

        # Saving data to memory
        if self.memory_length > 0:
            self.save_data()

        # Draw environment and agents
        if self.with_visualization:
            self.draw_frame()
            pygame.display.flip()

        # Moving time forward
        self.clock.tick(self.framerate)

    end_time = datetime.now()
    print(f"{datetime.now().strftime('%Y-%m-%d_%H-%M-%S.%f')} Total simulation time: ",
          (end_time - start_time).total_seconds())

    pygame.quit()
```

In a nutshell, we do the following:
* we create a loop that goes from zero until `T` (note `self.T` given that we are within a class)
* in each iteration we check if there is any interactive events in the pygame window and carry out some interactions accordingly (more about this later)
* Then we carry out collisions between the agents (if applicable)
* and update each agent according to the state of the world in the current timestep
* then we iterate again and update the visualization.
* at the end of the simulation we quit from pygame

Let's see an example simulation with 10 agents for 300 simulation timesteps. (Note that the pygame window sometimes pops up behind your active window)

In [1]:
# first we import our package so that we can use the classes that are defined there
from pygmodw25.sims import Simulation

# Creating a short test simulation instance with 15 agents and 150 timestep simulation time
test_simulation = Simulation(N=10, T=150)

# Start the simulation
test_simulation.start()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
Running simulation start method!
Starting main simulation loop!
2025-07-15_10-15-34.610425 Total simulation time:  6.070245


## Interactions
pygame is a game engine, and allows user interactions with your simulations. For your convenience we defined a few within the `interact_with_event` method of the `Simulation` class. These are:

**Keystrokes:**

- `f`aster: increase framerate (if your system allows)
- `s`lower: decrease framerate
- `d`efault: default framerate (25fps)
- `c`olor: turn on/off coloration according to agent orientation and velocity
- `space`: pause/continue simulation

**Cursor Events**:

- **move**: You can drag and drop agents around by clicking and holding them. This allows you to perturb the system without any need of coding.
- **rotate**: You can rotate the agents by first grabbing them and then using your mouse wheel (or scrolling event). In case you are using a laptop without a mouse, you can use the left and right arrows to rotate the agents **while** holding the agent with your left mouse button. Alternatively you can use scrolling **without** holding the agents with your left mouse button (first pause the simulation, then move your cursor above the agent and scroll without clicking).

Let's try a few of these here:


In [2]:
# first we import our package so that we can use the classes that are defined there
from pygmodw25.sims import Simulation

# Creating a short test simulation instance with 15 agents and 150 timestep simulation time
test_simulation = Simulation(N=10, T=300)

# Start the simulation
test_simulation.start()

Running simulation start method!
Starting main simulation loop!
Bye bye!


SystemExit: 

C:\Users\David\Desktop\PygameModelling25\venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Part I.: Modifying Agents and Simulation
Here comes the exciting part. We need to create our own agents. To do so we will use inheritance. If you don't know anything about inheritance, I suggest you read up on it here: https://www.programiz.com/python-programming/inheritance

For this workshop, the following information will suffice: It is one of the most important concepts in object-oriented programming. You can think of it, as one new class that you'd define will be copied from an old one and extended with new features. It is similar to biological inheritance, where the "child" often shares a lot of attributes with it's "parent", but also has new, unique features. Why is this beneficial? Because we don't have to reinvent the wheel every time we would like to add new features to our agents (or any other class). Instead, we simply inherit all basic features, and add a few new ones.

## Exercise 1: Inheritance
Let's see how it goes. Agent's in our framework are all initialized as blue circular pacman-like dots moving in straight lines passing through each other. Let's suppose we want these agents to have a **mood** attribute that can be either "neutral" or "angry". In the begining agents should be neutral, but, if any other agent approaches closer than 20 pixels, they should turn angry. To better visualize this change, I would like the agents to turn red from blue when they become angry, but turn back to blue once neutral again.

Here is how it goes:
* We create a new class `AngryAgent` that inherits from `AgentBase`, and change it's `__init__` method to include a new mood attribute (`self.mood`)
* We add a method in the class that switches the agent's mood and it's color according to other agents' distances
* We **override** the `update` method of the parent class, such that it includes our new behavioral rules instead of just passively moving straight.

Here we go:

In [1]:
# import base agent
from pygmodw25.agent import AgentBase
# import some predefined RGB colors from support
from pygmodw25 import support
import numpy as np

# Create new class inheriting from pygmodw25.agent.AgentBase
class AngryAgent(AgentBase):
    """
    Agent class realizing some neurotic agents that become angry with each other once too close. :(
    """
    # we recieve some attributes for the base agents and initiate the parent calss with them
    def __init__(self, id, radius, position, orientation, env_size, color, window_pad):
        """
        Initalization method of main agent class of the simulations

        Base parameters:

        :param id: ID of agent (int)
        :param radius: radius of the agent in pixels
        :param position: position of the agent bounding upper left corner in env as (x, y)
        :param orientation: absolute orientation of the agent (0 is facing to the right)
        :param env_size: environment size available for agents as (width, height)
        :param color: color of the agent as (R, G, B)
        :param window_pad: padding of the environment in simulation window in pixels
        """
        # Initializing supercalss (Base Agent)
        super().__init__(id, radius, position, orientation, env_size, color, window_pad)
        # Now we can add the new angry-specific attribute "mood"
        self.mood = "neutral"
    
    # let's now add the method that switches agents' mood and color accordingly
    def switch_mood(self, mood):
        self.mood = mood
        if self.mood == "neutral":
            self.color = support.BLUE
        elif self.mood == "angry":
            self.color = support.RED
    
    # Now let's modify the update method accordingly
    def update(self, agents):
        """Updates the state of the agent according to the position of other agents"""
        distances = []
        for ag in agents:
            if ag.id != self.id:
                # calculating distances
                dist = np.linalg.norm(np.array(self.position) - np.array(ag.position))
                distances.append(dist)

        # in case any other agent is closer than 20 pixels we get ANGRY!!!!
        if np.min(distances) < 20:
            self.switch_mood("angry")
        else:
            self.switch_mood("neutral")
        
        # After this modification we would write exactly the same as
        # what is in the parent (or super) class, so we can just reuse
        # these as follows instead
        super().update(agents)

pygame 2.6.1 (SDL 2.28.4, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


Looking good! Now we need to create a simulation that uses these new agents. Can you guess how we are gonna do that? We use inheritance! In the new `AngrySimulation` class we want to override the parent class's `add_new_agent` method, such that it now adds angry agents and not just the original vanilla ones. Note, that since we do not introduce new attributes to this new class, only a modified method, we do not have to change the `__init__`method. Let's go!

In [2]:
# import base simulation
from pygmodw25.sims import Simulation

# Create new class inheriting from pygmodw25.sims.Simulation
class AngrySim(Simulation):
    def add_new_agent(self, id, x, y, orient):
        """Adding a single NEUROTIC new agent into agent sprites"""
        agent = AngryAgent(
            id=id,
            radius=self.agent_radii,
            position=(x, y),
            orientation=orient,
            env_size=(self.WIDTH, self.HEIGHT),
            color=support.BLUE,
            window_pad=self.window_pad
        )
        self.agents.add(agent)


Now let's run an `AngrySim` instance with N=30 neurotic agents and run it for 300 time steps!

In [3]:
angry_sim_instance = AngrySim(N=30, T=300)
angry_sim_instance.start()

Running simulation start method!
Starting main simulation loop!
Bye bye!


SystemExit: 

C:\Users\David\Desktop\PygameModelling25\venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


They indeed sim angry when they get into traffic jams. Now let's make them swear too! To do so, we do the following:
* we create a new swearing agent type, that inherits from the angry agents
* we initialize this new swearing agent with some built in angry messages from which it can choose when becomes angry
* we add a `draw_message` method to the agent to visualize this random message once angry. Note that to visualize the message on the simulation screen (the window you see when running the simulation) you have to pass the `screen` attribute of the `Simulation` class to this class method.
* we then modify the simulation class such that it uses these new swearing type of agents
* and we add a visualization step to the simulation class's `draw_frame` method (that renders everything in each timestep), that calls the agents' `draw_message` method to also render agent messages

In [4]:
import random
import pygame
# import base simulation
from pygmodw25.sims import Simulation

# swearing agent inherits from angry agent which alredy has a mood swing, we just need to verbalizet this
class SwearingAngryAgent(AngryAgent):
    def __init__(self, id, radius, position, orientation, env_size, color, window_pad):
        super().__init__(id, radius, position, orientation, env_size, color, window_pad)
        # Get some angry little messages
        self.angry_messages = [
            "What the heck!", "Are you blind?!", "Move it!", "Get lost!", "Seriously?!", "Watch it!", "Damn it!", "Zack Zack!"
        ]
        # choose one for now
        self.current_message = random.choice(self.angry_messages)
        
    def draw_message(self, screen):
        '''Drawing an angry message if it is called and the agent is angry. Otherwise it chooses another random message'''
        if self.mood == "angry":
            # creating text rendering
            font = pygame.font.Font(None, 15)
            text_surface = font.render(self.current_message, True, (255, 50, 50))
            text_rect = text_surface.get_rect()
            text_rect.center = (int(self.position[0]) + 2*self.radius, int(self.position[1]) + 2*self.radius+10)
            
            # render text on the simulation screen that is passed
            screen.blit(text_surface, text_rect)
        else:
            # choose random message if not angry at the moment
            self.current_message = random.choice(self.angry_messages)
            

# Create new class inheriting from pygmodw25.sims.Simulation
class AngrySim(Simulation):
    def add_new_agent(self, id, x, y, orient):
        """Adding a single SWEARING NEUROTIC new agent into agents of simulation"""
        agent = SwearingAngryAgent(
            id=id,
            radius=self.agent_radii,
            position=(x, y),
            orientation=orient,
            env_size=(self.WIDTH, self.HEIGHT),
            color=support.BLUE,
            window_pad=self.window_pad,
        )
        self.agents.add(agent)
    
    def draw_frame(self):
        """Drawing environment, agents and every other visualization in each timestep"""
        # almost everything is the same as in the original SImulation class, but we extend it
        super().draw_frame()
        # extending rendering with one additional step, calling the agents' swearing method
        for ag in self.agents:
            ag.draw_message(self.screen)
        
angry_sim_instance = AngrySim(N=30, T=300)
angry_sim_instance.start()

Running simulation start method!
Starting main simulation loop!
Bye bye!


SystemExit: 

Wow they are really angry now! At this point you know how to:
* Navigate in the provided code base (AgentBase and Simulation classes, class attributes and methods, update and start methods)
* Use the concept of inheritance for your purposes:
    * Modify the agent and simulation classes (through inheritance) to use their base features but add new behavior to them.
    * Run simulations that use these new features.
    
After this part let's use our knowledge for actual science.

# Part II.: The Three-Zone-Model

Here, we will explore an agent-based model for simulating collective dynamics of animals (e.g. fish schools) through the provided package and small tasks. Feel free to provide your answers in this notebook. Our main task is the exploration of collective dynamics in a 2D model, related to the Three-Zone-Model (Two-Zone-Models) [by Couzin et al.](https://www.sciencedirect.com/science/article/pii/S0022519302930651), where the interaction between individuals is governed by three basic interactions: long-range attraction, short-ranged repulsion and alignment. 

## Brief Introduction
### Fundamental model

The code solves a set of (stochastic) differential equation describing a set of $N$ interacting agents ($i= 1,\dots, N$). The dynamics of each agent (in 2d) is described by the following equations of motion:

$$ \frac{d \vec{r}_i}{dt}=\vec{v}_i(t) $$
$$ \vec{v}_i(t) = {s_i\cos(\varphi_i(t)) \choose s_i\sin(\varphi_i(t)) } $$
$$ \frac{d \varphi_i}{dt} = \frac{1}{s_i}\left( F_{i,\varphi} + \eta_{i,\varphi} \right) $$


Here $\vec{r}_i$, $\vec{v}_i$ are the Cartesian position and velocity  vectors of the focal agent, wth $s_i$ being the (constant) speed of agent $i$. Furthermore, $\eta_{i,\varphi}$ represents Gaussian white noise introducing randomness in the motion of individuals, and $\vec{F}_{i,\varphi}$ is the projections of the total social force inducing a turning behavior.
$$ F_{i,\varphi}=\vec{F}_i \cdot \vec{u}_{\varphi,i} = \vec{F}_i {- s_i\sin\varphi_i \choose s_i\cos\varphi_i } $$


The total effective social force is a sum of three components:
$$ \vec{F}_i=\vec{F}_{i,rep}+\vec{F}_{i,alg}+\vec{F}_{i,att} $$


**Attraction:
$$\vec{F}_{i,att}=\sum_{j \in Neigh} +\mu_{att}S_{att}({r}_{ji}) \hat{r}_{ji} $$
Repulsion:
$$\vec{F}_{i,rep}=\sum_{j \in Neigh} -\mu_{rep}S_{rep}({r}_{ji}) \hat{r}_{ji}$$
Alignment:**
$$\vec{F}_{i,alg}=\sum_{j \in Neigh} \mu_{alg}S_{alg}({r}_{ji}) (\vec{v}_j-\vec{v}_i)$$
with $\hat r = \vec{r}/|r|$.

The strength of the different interactions is set by a constant $\mu_X$ and a sigmoid function of distance, which goes from 1 to 0, with the transition point at $r_{X,0}$ and steepness $a_{X}$:
$$ S_X(r)=\frac{1}{2}\left(\tanh(-a(r-r_{X,0})+1\right) $$

<img src="data/images/scheme_ranges.png" width='800'>

**Figure1.:** Local interaction forces around an agent

<img src="data/images/int_ranges.png">

**Figure2.:** Example of the 3 interaction zones around a focal agent

## Task 1.: Build a basic `ZonalAgent`
Our first task today is to build an agent that acts according to these rules, i.e. according to certain distances (or zones) around them. The above three simple rules can be formulated in python syntax by checking the distance between individual agents and acting according to these.

**Specification**:
* Create a new `ZonalAgent` class inheriting from the `AgentBase`class
* modify it's `__init__` method such that it accepts the parameters of the flocking model as class attributes with the following default values, that is:
    * Interaction strengths: Attraction (self.s_att = 0.02), Repulsion (self.s_rep = 5), Alignment (self.s_alg = 8)
    * Interaction ranges (Zones) and stepness: self.steepness_att = -0.5, self.r_att = 250, self.steepness_rep = -0.5, self.r_rep = 50, self.steepness_alg = -0.5, self.r_alg = 150
    * Noise: self.noise_sig = 0.1
* Create an `update_forces()` method in this class that implements the above rules. Hints below.
* modify the agent's `update` method such that it calls this new `update_forces()` method to update the agents heading before rendering it.

**Definition of Done**: Your agents act according to the above model and are able to move in a flock to a shared direction.

## 🧭 Step-by-Step Guide: Implementing `update_forces()` for a Zonal Flocking Model

To achieve the above goals, you must update each agent's desired **velocity** (`self.dv`) and **rotation** (`self.dtheta`) based on the position and movement of nearby agents, using three key types of social forces: **attraction**, **repulsion**, and **alignment**. Here’s how to approach it: Add an `update_forces()` method to this class, and implement the following steps:

---
### **1. Understand the Agent’s Heading vector (`heading_vec`)**

To determine the direction the agent is facing, you need a vector that points from the **center of the agent** to the **edge of its body** in the direction of its current orientation within the environment (`self.orientation`).

Here’s how to think about it:

- The agent is represented as a circle, but `self.position` gives the **top-left corner** of its bounding box, not the center.
- So first, compute the **center position** of the agent (`v1_s_x`, `v1_s_y`):
- Then, calculate a point on the **edge of the circle** in the direction the agent is facing (`v1_e_x`, `v1_e_y`):
  - Use the variation of `cos(self.orientation)` for the x-direction, and `sin(self.orientation)` for the y-direction.
  - Multiply these by the **actual radius** of the agent (since it's not a unit circle)
  - (Note: y is subtracted due to screen coordinates increasing downward.)
- The difference between this edge point and the center gives you a **heading vector**, which describes where the agent is currently facing:

```python
  v1_x = v1_e_x - v1_s_x
  v1_y = v1_e_y - v1_s_y
  heading_vec = np.array([v1_x, v1_y])
```

🔄 This vector doesn't define velocity or movement. It's purely about orientation and is later used to determine how closely the agent’s actual direction aligns with the direction of a social force, so it will allow us to calculate the change of an agent's heading according to the above model.

---

### **2. Prepare Accumulators for Social Forces**

- Initialize the following vectors:
  - `vec_attr_total = np.zeros(2)` → for **attraction**
  - `vec_rep_total = np.zeros(2)` → for **repulsion**
  - `vec_alg_total = np.zeros(2)` → for **alignment**
- These will store the total force contributions from all other agents.

---

### **3. Loop Through All Other Agents (`for ag in agents`)**

- Skip the agent itself (`if ag.id != self.id:`).
- For each neighbor:
  - Calculate its **center position** (`ag_pos_x`, `ag_pos_y`).
  - Calculate the focal agent’s own center (`v1_s_x`, `v1_s_y` in the previous step).
  - Compute the **distance vector** `distvec` between the two with simple subtractions.
  - Use either Euclidean distance or `support.distance_infinite()` depending on the boundary condition (`self.boundary`).

---

### **4. Calculate Velocity Differences (`dvel`)**

- Compute the velocity vectors of the focal agent (`s_vel`) and the other agent (`ag_vel`), using their orientation (self.orientation) and speed (self.velocity).
- Subtract to get the **velocity difference**: `dvel = ag_vel - s_vel`.

---

### **5. Calculate Individual Social Forces**

- For each neighbor, use the appropriate helper function to compute the force based on `distvec` (and `dvel` for alignment):
  - `support.CalcSingleAttForce(self.r_att, self.steepness_att, distvec)`
  - `support.CalcSingleRepForce(self.r_rep, self.steepness_rep, distvec)`
  - `support.CalcSingleAlgForce(self.r_alg, self.steepness_alg, distvec, dvel)`
- Add each result to the corresponding total vectors `vec_attr_total`, `vec_rep_total`, `vec_alg_total`.
- Explore these helper functions within the support file. What are these?

---

### **6. Combine the Forces into One (`force_total`)**

- Combine the three total force vectors using the zone-specific strengths. Note the signs!

---

### **7. Convert the Force into a Speed (`vel`) and Turning Angle (`theta`)**

- Compute the desired **speed** as a function of the force magnitude:
  ```python
  vel = self.v_max * np.linalg.norm(force_total)
  ```
- Compute the **angle** between the agent’s current heading (`heading_vec`) and the new desired direction (`force_total`):
  ```python
  closed_angle = support.angle_between(heading_vec, force_total)
  closed_angle = closed_angle % (2 * np.pi)
  ```
- Convert `closed_angle` to your simulation's turning convention (`theta`):
  - Adjust it to be in `[-π, π]` rather than `[0, 2π]`
  - Make sure signs and directions match your orientation system (e.g., 0 radians = facing right)

---

### **8. Add Directional Noise (Optional)**

- If `self.noise_sig > 0.0`, add a small normally distributed value to `theta`

---

### **9. Store Results for the Next Update Step**

- Set the agent’s orientation change (`self.dtheta`) and velocity change (`self.dv`) based on your calculations:
  ```python
  self.dtheta = theta
  self.dv = vel
  ```
  
  
**Please modify the code below at the dedicated lines to solve the task accordingly.**

In [1]:
import numpy as np
from pygmodw25.agent import AgentBase

class ZonalAgent(AgentBase):
    """
    Agent class that includes all private parameters of the agents and all methods necessary to move in the environment
    and to make decisions.
    """

    def __init__(self, id, radius, position, orientation, env_size, color, window_pad):
        """
        Initalization method of main agent class of the simulations

        :param id: ID of agent (int)
        :param radius: radius of the agent in pixels
        :param position: position of the agent bounding upper left corner in env as (x, y)
        :param orientation: absolute orientation of the agent (0 is facing to the right)
        :param env_size: environment size available for agents as (width, height)
        :param color: color of the agent as (R, G, B)
        :param window_pad: padding of the environment in simulation window in pixels
        """
        # Initializing supercalss (Base Agent)
        super().__init__(id, radius, position, orientation, env_size, color, window_pad)

        # Specifying parameters for this special 3-zone agent
        # Interaction strength
        # Attraction
        self.s_att = 0.02
        # Repulsion
        self.s_rep = 5
        # Alignment
        self.s_alg = 8

        # Interaction ranges (Zones)
        # Attraction
        self.steepness_att = -0.5
        self.r_att = 250
        # Repulsion
        self.steepness_rep = -0.5
        self.r_rep = 50
        # Alignment
        self.steepness_alg = -0.5
        self.r_alg = 150

        # Noise
        self.noise_sig = 0.1

    def update_forces(self, agents):
        """Updateing overall social forces on agent according to velocity and distance of others"""
        # CALCULATING change in velocity and orientation in the current timestep
        # vel, theta = support.random_walk()
        # center point
        v1_s_x = self.position[0] + self.radius
        v1_s_y = self.position[1] + self.radius

        # point on agent's edge circle according to it's orientation
        v1_e_x = self.position[0] + (1 + np.cos(self.orientation)) * self.radius
        v1_e_y = self.position[1] + (1 - np.sin(self.orientation)) * self.radius

        # vector between center and edge according to orientation
        v1_x = v1_e_x - v1_s_x
        v1_y = v1_e_y - v1_s_y

        heading_vec = np.array([v1_x, v1_y])

        # CALCULATING attraction force with all agents:
        vec_attr_total = np.zeros(2)
        vec_rep_total = np.zeros(2)
        vec_alg_total = np.zeros(2)
        for ag in agents:
            if ag.id != self.id:
                # Distance between focal agent and given pair
                ag_pos_x = ag.position[0] + ag.radius
                ag_pos_y = ag.position[1] + ag.radius
                s_pos_x = self.position[0] + self.radius
                s_pos_y = self.position[1] + self.radius
                if self.boundary == "bounce_back":
                    distvec = np.array([ag_pos_x - s_pos_x, ag_pos_y - s_pos_y])
                elif self.boundary == "infinite":
                    distvec = support.distance_infinite(np.array([s_pos_x, s_pos_y]),
                                                        np.array([ag_pos_x, ag_pos_y]))

                # Difference between velocity between given agents
                s_vel = np.array([self.velocity * np.cos(self.orientation), - self.velocity * np.sin(self.orientation)])
                ag_vel = np.array([ag.velocity * np.cos(ag.orientation), - ag.velocity * np.sin(ag.orientation)])
                dvel = ag_vel - s_vel

                # Calculating interaction forces
                vec_attr_total += support.CalcSingleAttForce(self.r_att, self.steepness_att, distvec)
                vec_rep_total += support.CalcSingleRepForce(self.r_rep, self.steepness_rep, distvec)
                vec_alg_total += support.CalcSingleAlgForce(self.r_alg, self.steepness_alg, distvec, dvel)

        force_total = self.s_att * vec_attr_total - self.s_rep * vec_rep_total + self.s_alg * vec_alg_total

        vel = self.v_max * np.linalg.norm(force_total)
        closed_angle = support.angle_between(heading_vec, force_total)
        closed_angle = (closed_angle % (2 * np.pi))
        # at this point closed angle between 0 and 2pi, but we need it between -pi and pi
        # we also need to take our orientation convention into consideration to recalculate
        # theta=0 is pointing to the right
        if not np.isnan(closed_angle):
            if 0 < closed_angle < np.pi:
                theta = -closed_angle
            else:
                theta = 2 * np.pi - closed_angle
        else:
            theta = 0

        # Adding directional noise
        if self.noise_sig > 0.0:
            noiseP = np.random.normal(0.0, self.noise_sig, size=1)
            theta += noiseP[0]

        self.dtheta = theta
        self.dv = vel
        
    def update(self, agents):
        self.update_forces(agents)
        super().update(agents)

pygame 2.6.1 (SDL 2.28.4, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


Now try to use your new agent in a simulation by running the cell below:

In [2]:
from pygmodw25.sims import Simulation
from pygmodw25 import support
# Changing simulation such that we use ZonalAgents
class ZonalSim(Simulation):
    def add_new_agent(self, id, x, y, orient):
        """Adding a single ZONAL new agent into agent sprites"""
        agent = ZonalAgent(
            id=id,
            radius=self.agent_radii,
            position=(x, y),
            orientation=orient,
            env_size=(self.WIDTH, self.HEIGHT),
            color=support.BLUE,
            window_pad=self.window_pad,
        )
        self.agents.add(agent)
        
zonal_sim_instance = ZonalSim(N=12, T=300)
zonal_sim_instance.start()

Running simulation start method!
Starting main simulation loop!
Bye bye!


SystemExit: 

C:\Users\David\Desktop\PygameModelling25\venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Task 2: Exploring Zones of Agents

Use the code snippet below to run the simulation for only a pair of agents.

We turn on the visualization of the local interaction zones by pressing setting the simulation's `show_zones` attribute to 1. If you get the printed message "Error while drawing agent zones!" you might have not been keeping the naming of the attributes in the above desrciption.

The long-range attraction zone is denoted with a green, the intermediate alignment zone with a yellow and the short-range repulsion with a red circle around the agents. Without pausing the simulation hold one of the agents still with your cursor.  

1. How does the other agent react? Is there anything surprising or different than what you expected?
2. Why? How can you explain what you see with the effect of the 3 zones (attraction/alignment/repulsion)?
3. What happens if you also start rotating the agent at the same time you hold it? Which zone is responsible for the temporary change in the behavior? 

In [ ]:
zonal_sim_instance = ZonalSim(N=2, T=2500)
zonal_sim_instance.show_zones = 1
zonal_sim_instance.start()

## Task 3: Understanding Flocking Parameters

As we have seen in the introduction to the zonal model each agent has 3 zones of local interaction parametrized by their interaction ranges, steepness parameters and interaction strengths.

In the next section we will see how systematically changing these parameters will influence the collective behavior of the system.

In [ ]:
zonal_sim_instance = ZonalSim(N=12, 
                              T=600, 
                              width=500,  # Arena width in pixels
                              height=500,  # Arena height in pixels
                              agent_radius=10,
                              physical_obstacle_avoidance=True)

# we loop through all the agents of the created simulation and individually change their parameters
print("Setting parameters for agent", end = ' ')
for agent in zonal_sim_instance.agents:
    print(f"{agent.id}", end = ', ')
    
    # changing angular noise (sigma)
    agent.noise_sig = 0.1
    
    # changing their default flocking parameters
    agent.s_att = 0.02  # attraction strength (AU)
    agent.s_rep = 5  # repulsion strength (AU)
    agent.s_alg = 10  # alignment strength (AU)

    agent.r_att = 200  # attraction range (px)
    agent.r_rep = 50  # repulsion range (px)
    agent.r_alg = 100  # alignment range (px)
    
    agent.steepness_att = -0.5  # steepness in attraction force calculation (sigmoid)
    agent.steepness_rep = -0.5  # steepness in repulsion force calculation (sigmoid)
    agent.steepness_alg = -0.5  # steepness in alignment force calculation (sigmoid)
    
    # changing maximum velocity and simulation timesteps
    agent.v_max = 2
    agent.dt = 0.05
    
    agent.boundary = "bounce_back"
    
    
# Now we can start the simulation with the changed agents
zonal_sim_instance.start()

Use the example code snippet above and modify it to answer the following questions. 

1. **Individual Dynamics**: Turn off all the interaction forces. Perform simulations with different angular noise values (noise_sig) and explore the behavior of the agents (you can try: 0.1, 1, 3). For the next tasks fix this parameter to 0.1.
2. **Obstacle Avoidance**:
    - a. Re-introduce a strong local repulsion interaction (`s_rep = 5`) and by that implement obstacle avoidance in a group of 10 agents. Set the repulsion steepness to -1.
    - b. Test the repulsion beahvior with different repulsion ranges (20, 50, 150). Feel free to move around the agents and see what happens when you move them in each others' repulsion zone.
    - c. **Hint:** You can visualize the zones around the agents with `z`. Only those zones will be shown for which the corresponding interaction strength is larger than zero. 
    - d. Explore the effect of the repulsion steepness (-0.1, -0.5, -1). When does repulsion take place with low or high steepness parameter? 
    - e. Do you think this obstacle avoidance will always avoid agents from collision?
    - f. For the upcoming experimentation fix repulsion strength to 5, steepness to -0.5 and range to 50
 
3. **Attraction-Repulsion**
    - a. Re-intorduce attraction and explore the beahavior with different attraction strenths 0.05, 0.5, 1, 5.
    - b. What happens for attraction strength of 0.05 and 5? What would you consider a realistic attraction strength for a mosquito swarm?
    - c. For the next experiment fix the attraction strength to 0.02, attraction range to 200 and steepness to -0.5.
    
4. **Movement Coordination**
    - a. What do you think you will observe when you introduce an intermediate zone where agents align to their neighbors?
    - b. Re-introduce the alignment zone by setting a strong alignment strength of 5. Is your observation matching with what you expected? What happens if you increase the attraction strength to 2?
    - c. Change back the attraction strength to 0.02. Introduce repellent walls by writing the following in the loop: `agent.boundary = "bounce_back"`. Is the system robust enough to handle such perturbations? Try different alignment strengths of 1.75 and 5. What is the difference? Try different angular noise values as well.

## Task 4.: Heterogeneity (Optional/Home)
Now that you know how to change the flocking parameters of the agents we can introduce heterogeneity. 

Compared to fully idealized model systems, natural groups are heterogenous in terms of some property. Let's suppose in our flock some of the individuals are faster than others.

1. Copy the above code snippet and paste it into a new cell in order to model a swarm of 20 agents from which half is 10% faster than the other half? You can control the agents' speed with their `v_max` attribute. (**Hint:** You can change the color of the fast agents by setting their `orig_color` attribute to any RGB tuple `(R, G, B)` where R, G, B are integers between 0 and 255).
2. Do you see any effect of such a heterogeneity?
3. What happens when the fast agents are twice as fast as the slow agents (e.g. `v_max=2` and `v_max=1` respectively)? What do you see?
4. Heterogeneity in natural systems are usually on a spectrum and the difference is not binary (fast/slow). Set the maximum velocity of agents in the group as a random uniform distribution between 1 and 2.5. Set the color of the agents in a way that they give useful information about their maximum speed.
5. You can use 30 agents and set the arena to 700x700. Set the boundary condition of the agents to `"bounce_back"` so that they can not cross walls.
6. What do you observe?
7. Is the obstacle avoidance (strong repulsion force) always successful in large groups? Pygame provides useful implementation of [collision groups](https://stackoverflow.com/questions/29640685/how-do-i-detect-collision-in-pygame). Turn on pygame-based obstacle avoidance by adding `physical_obstacle_avoidance=True` in the argment when creating your `Simulation` class instance. Did anything change in the effect you have observed in 4.6.?

## Task 5.: Modelling epidemic spread of disease with the SIR model (Optional/Home)

In the [SIR model](https://de.wikipedia.org/wiki/SIR-Modell) of disease spread agents have 3 basic states of infection. When agents have not yet contacted the disease they are susceptible for a possible infection ($S$). When they get infected ($I$) they can spread the disease to other susceptible agents in a given infection radius ($R_I$) with probability ($P_I$). In our immplementation, agents can only come out from an infected state 2 way (note that this can vary across model implementations). They either recover ($R$) with probability $P_R$ and get immune to future infections, or they die ($D$) with probability $P_D$)

**Task:**
Implement this model by extending the `AgentBase` class:
* Each agent should have the SIR model parameters as attributes, such as probabilities for infection, recovery and death
* They should have a `state` attribute being one of the following 4 states: S, I, R or D
* Override the `update`method such that it includes a probabilistic state change according to the above rules.
* If agents are susceptible, they should be blue
* Infected agents should turn red
* Recovered agents are green
* Agents that are dead should stop moving and turn grey.

Creat a new `SIRSimulation` class that uses these agents and start the simulation. 

**Optional/Pro:** 
* Implement data saving, by overriding the `BridgeIO` method of the `SIRSimulation` class, such that
    * In each simulation step the `SIRSimulation` class saves internally how many agents of different state there are within the simulation.
    * In the last timestep it writes these arrays into files
* Use matplotlib to plot these and see how the different probabilities change the model outcomes. They should look similar to this plot:

<img src="data/images/SIREvolution.png">

**Figure3.:** Example of the 3 interaction zones around a focal agent

# Assignment
During the next session we will connect the flocking model we created to the mixed-reality system at our cluster, called CoBeXR (https://www.biorxiv.org/content/10.1101/2025.06.15.659521v1). This system creates a loop from tracked human action - through simulation - to visualization. In sum, it allows you to track an embodied physical object, e.g. a tracking cane, and change a simulated world according to its movement, which then in real time projected back to the arena. This allows a quick experimentation with computational models among many other applications.

To connect your model with the system, on the other hand, you will have to connect it to CoBeXR through file read and write commands (so called IO Bridge). This concept is very simple, you can think of it, as the tracking system writes the position of the tracked object into a file. Your simulation needs to read it, interpret it and change the simulation accordingly in every timestep.

**For example:** You can set up the system to track your hand and write it's 3D position into a selected file. Your simulation could, for instance read this file and according to it's Z coordinate, change the movement noise of the agents (`agent.noise_sig` between 0 and 1). As a result, moving the tracked object in the arena up and down (in this case your hand), would allow a real-time experimentation of the change of this parameter.

Your task is to create such an IO Bridge so we can connect your model to the system. You are free to think of what would you like to change in the model according to the tracked object's physical position. Examples can be the actual position of one of the agents, system parameters, such as agents' speed, behavioral parameters such as attrcation strength, visualization, such as colors, etc. For those who want more challenge and fun, you can also implement the SIR model (as above) and control infection parameters, or create a super-infectious agent that follows the tracked object (getting close to gamification).

To create the IOBridge, please create a new `CoBeSimulation` class (inheriting from `ZonalSim`, or the SIR simulation for advanced case) and without any further change you have to override it's `bridgeIO` method. You can find the pseudocode below

```python
import json

class CoBeSimulation(ZonalSim):
    def bridgeIO(self):
        ### DEFINE THE PATH OF THE FILE TO BE READ
        
        ### READ JSON FILE HERE
            ## check if file exists and accessible
            ## if yes, open and read
        
        ### INTERPRET THE READ VALUE HERE
        ### AND APPLY CHANGES TO AGENTS/SIMULATION PARAMETERS
```

**Specification:** 
* The file to be read is a json file
* The location of the file is the working folder and has a name `bridge.json`
* The file contains a list, where each element has an ID key containing the ID of the tracked objects and its x, y, z position of the tracked object in the arena.
* You can have zero, one or many objects tracked (optional, must work for one or zero)
* The content of an example file looks like this:

```json
[{"ID": 1, "x": 1391, "y": -4166, "z": 1249}]
```

* the maximum absolute coordinates both in x and y are 3500mm (can be +-)
* the maximum coordinates in z is expcted to be around 3000mm
* in case you get larger numbers truncate these to the maximum (or minimum) values

**Definition of Done:**
Your simulation changes a single parameter visibly when the file has been changed during simulation run.

In [8]:
# you assignment here

In [7]:
import pygame
cobe_sim = CoBeSimulation(N=10, T=500)
cobe_sim.show_zones = 1
cobe_sim.initiate_mixed_reality()
try:
    cobe_sim.start()
except:
    pygame.quit()

Running simulation start method!
Starting main simulation loop!
Bye bye!
